In [1]:
import pandas as pd
import numpy as np 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import model_selection 
import seaborn as sns 
from sklearn.model_selection import StratifiedKFold 
from sklearn import metrics
from numpy import mean
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import os

In [9]:
z = os.path.split("Dataset/abc/JM1.csv")
z[1]

'JM1.csv'

In [45]:
df = pd.read_csv("Dataset/JM1.csv",index=False)
df['label'] = df['label'].map({"b'N'" :False,"b'Y'" :True})
df

TypeError: read_csv() got an unexpected keyword argument 'index'

In [42]:
df.iloc[:,:-1]

,LOC_BLANK,BRANCH_COUNT,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,LOC_EXECUTABLE,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,...,HALSTEAD_ERROR_EST,HALSTEAD_LENGTH,HALSTEAD_LEVEL,HALSTEAD_PROG_TIME,HALSTEAD_VOLUME,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,LOC_TOTAL
0,447.0,826.0,12.0,157.0,470.0,385.0,113.0,2824.0,210.28,384.45,...,26.95,8441.0,0.00,1726654.57,80843.08,3021.0,5420.0,609.0,155.0,3442.0
1,37.0,29.0,8.0,42.0,19.0,19.0,6.0,133.0,108.14,46.32,...,1.67,685.0,0.02,12891.31,5009.32,295.0,390.0,121.0,38.0,222.0
2,11.0,405.0,0.0,17.0,404.0,2.0,1.0,814.0,101.20,206.01,...,6.95,2033.0,0.00,238607.05,20848.47,813.0,1220.0,811.0,411.0,844.0
3,106.0,240.0,7.0,344.0,127.0,105.0,33.0,952.0,218.17,215.17,...,15.65,5669.0,0.00,561159.25,46943.69,2301.0,3368.0,262.0,49.0,1411.0
4,101.0,464.0,11.0,75.0,263.0,256.0,140.0,1339.0,106.50,337.36,...,11.98,4308.0,0.00,673377.60,35928.07,1556.0,2752.0,226.0,98.0,1532.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9586,2.0,7.0,0.0,0.0,4.0,4.0,1.0,13.0,32.93,7.33,...,0.08,52.0,0.14,98.38,241.48,22.0,30.0,15.0,10.0,18.0
9587,2.0,3.0,0.0,0.0,2.0,2.0,1.0,5.0,15.72,8.25,...,0.04,30.0,0.12,59.43,129.66,11.0,19.0,8.0,12.0,9.0
9588,10.0,7.0,0.0,1.0,4.0,2.0,1.0,29.0,19.68,26.40,...,0.17,103.0,0.04,762.04,519.57,44.0,59.0,15.0,18.0,42.0
9589,2.0,1.0,0.0,0.0,1.0,1.0,1.0,6.0,17.44,8.44,...,0.05,36.0,0.12,68.98,147.15,15.0,21.0,8.0,9.0,10.0


In [31]:
def featureSelection(df):
        best_features = SelectKBest(score_func=chi2,k=10)
        fit = best_features.fit(df.iloc[:,:-1],df.iloc[:,-1])

        dfscores = pd.DataFrame(fit.scores_)
        df_cols = pd.DataFrame(df.iloc[:,:-1].columns)

        feature_scores = pd.concat([df_cols,dfscores],axis=1)
        feature_scores.columns = ['parameters','score']
        selected_features = feature_scores.nlargest(10,'score').parameters.values
        df = df.drop(selected_features, axis=1)
        print(df.head())
        return selected_features


In [35]:

def normalize(df):
    result = df.copy()
    for col in df.iloc[:,:-1].columns:
        max_value = df[col].max()
        min_value = df[col].min()
        result[col] = (df[col] - min_value) / (max_value - min_value)
    print(df.head())
    return result

def preprocessing(df):
    # select k best columns
    best_features = SelectKBest(score_func=chi2,k=10)
    fit = best_features.fit(df.iloc[:,:-1],df.iloc[:,-1])

    dfscores = pd.DataFrame(fit.scores_)
    df_cols = pd.DataFrame(df.iloc[:,:-1].columns)

    feature_scores = pd.concat([df_cols,dfscores],axis=1)
    feature_scores.columns = ['parameters','score']
    selected_features = feature_scores.nlargest(10,'score').parameters.values
    df = df.drop(selected_features, axis=1)
    # normalize the data
    df = normalize(df)
    return df


In [47]:
df = preprocessing(df)

   LOC_BLANK  LOC_CODE_AND_COMMENT  LOC_COMMENTS  CYCLOMATIC_COMPLEXITY  \
0      447.0                  12.0         157.0                  470.0   
1       37.0                   8.0          42.0                   19.0   
2       11.0                   0.0          17.0                  404.0   
3      106.0                   7.0         344.0                  127.0   
4      101.0                  11.0          75.0                  263.0   

   DESIGN_COMPLEXITY  ESSENTIAL_COMPLEXITY  HALSTEAD_CONTENT  \
0              385.0                 113.0            210.28   
1               19.0                   6.0            108.14   
2                2.0                   1.0            101.20   
3              105.0                  33.0            218.17   
4              256.0                 140.0            106.50   

   HALSTEAD_DIFFICULTY  HALSTEAD_ERROR_EST  HALSTEAD_LEVEL  \
0               384.45               26.95            0.00   
1                46.32                1.

In [37]:
df

,LOC_BLANK,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_ERROR_EST,HALSTEAD_LEVEL,NUM_UNIQUE_OPERATORS,label
0,1.000000,0.111111,0.456395,1.000000,0.957606,0.682927,0.369055,0.919297,1.000000,0.00,0.377129,True
1,0.082774,0.074074,0.122093,0.038380,0.044888,0.030488,0.189793,0.110760,0.061967,0.02,0.092457,True
2,0.024609,0.000000,0.049419,0.859275,0.002494,0.000000,0.177612,0.492611,0.257885,0.00,1.000000,True
3,0.237136,0.064815,1.000000,0.268657,0.259352,0.195122,0.382902,0.514515,0.580705,0.00,0.119221,True
4,0.225951,0.101852,0.218023,0.558635,0.635910,0.847561,0.186914,0.806695,0.444527,0.00,0.238443,True
...,...,...,...,...,...,...,...,...,...,...,...,...
9586,0.004474,0.000000,0.000000,0.006397,0.007481,0.000000,0.057794,0.017527,0.002968,0.14,0.024331,False
9587,0.004474,0.000000,0.000000,0.002132,0.002494,0.000000,0.027590,0.019727,0.001484,0.12,0.029197,False
9588,0.022371,0.000000,0.002907,0.006397,0.002494,0.000000,0.034540,0.063128,0.006308,0.04,0.043796,False
9589,0.004474,0.000000,0.000000,0.000000,0.000000,0.000000,0.030608,0.020182,0.001855,0.12,0.021898,False


In [38]:
X = df.values

In [40]:
X.shape

(9591, 12)

In [48]:
df

,LOC_BLANK,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_ERROR_EST,HALSTEAD_LEVEL,NUM_UNIQUE_OPERATORS,label
0,1.000000,0.111111,0.456395,1.000000,0.957606,0.682927,0.369055,0.919297,1.000000,0.00,0.377129,True
1,0.082774,0.074074,0.122093,0.038380,0.044888,0.030488,0.189793,0.110760,0.061967,0.02,0.092457,True
2,0.024609,0.000000,0.049419,0.859275,0.002494,0.000000,0.177612,0.492611,0.257885,0.00,1.000000,True
3,0.237136,0.064815,1.000000,0.268657,0.259352,0.195122,0.382902,0.514515,0.580705,0.00,0.119221,True
4,0.225951,0.101852,0.218023,0.558635,0.635910,0.847561,0.186914,0.806695,0.444527,0.00,0.238443,True
...,...,...,...,...,...,...,...,...,...,...,...,...
9586,0.004474,0.000000,0.000000,0.006397,0.007481,0.000000,0.057794,0.017527,0.002968,0.14,0.024331,False
9587,0.004474,0.000000,0.000000,0.002132,0.002494,0.000000,0.027590,0.019727,0.001484,0.12,0.029197,False
9588,0.022371,0.000000,0.002907,0.006397,0.002494,0.000000,0.034540,0.063128,0.006308,0.04,0.043796,False
9589,0.004474,0.000000,0.000000,0.000000,0.000000,0.000000,0.030608,0.020182,0.001855,0.12,0.021898,False


In [49]:
train,test = train_test_split(df,test_size=0.1)

In [50]:
train

,LOC_BLANK,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_ERROR_EST,HALSTEAD_LEVEL,NUM_UNIQUE_OPERATORS,label
2186,0.008949,0.000000,0.008721,0.010661,0.004988,0.000000,0.033803,0.040985,0.004082,0.06,0.031630,False
2665,0.008949,0.000000,0.000000,0.006397,0.002494,0.012195,0.036453,0.065471,0.007050,0.04,0.046229,False
6054,0.013423,0.000000,0.005814,0.021322,0.014963,0.000000,0.036453,0.094165,0.010019,0.03,0.051095,False
5373,0.031320,0.009259,0.020349,0.042644,0.027431,0.036585,0.117554,0.057461,0.020037,0.04,0.051095,False
4109,0.002237,0.000000,0.000000,0.008529,0.002494,0.024390,0.020464,0.045744,0.002597,0.05,0.041363,False
...,...,...,...,...,...,...,...,...,...,...,...,...
5051,0.006711,0.000000,0.000000,0.004264,0.002494,0.000000,0.022816,0.031612,0.002226,0.08,0.034063,False
4998,0.004474,0.009259,0.011628,0.021322,0.007481,0.000000,0.068693,0.045433,0.009276,0.05,0.034063,False
2987,0.002237,0.000000,0.000000,0.000000,0.000000,0.000000,0.038471,0.013319,0.001484,0.18,0.014599,False
4871,0.006711,0.000000,0.008721,0.004264,0.004988,0.000000,0.100706,0.014586,0.004453,0.16,0.019465,False


In [51]:
test

,LOC_BLANK,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_ERROR_EST,HALSTEAD_LEVEL,NUM_UNIQUE_OPERATORS,label
7933,0.004474,0.000000,0.017442,0.006397,0.000000,0.000000,0.020657,0.050215,0.002968,0.05,0.043796,False
5957,0.020134,0.000000,0.000000,0.000000,0.000000,0.000000,0.084489,0.036179,0.008905,0.07,0.029197,False
510,0.013423,0.009259,0.011628,0.004264,0.004988,0.000000,0.073186,0.027810,0.005937,0.09,0.031630,True
2682,0.002237,0.000000,0.017442,0.023454,0.004988,0.012195,0.054109,0.044189,0.007050,0.05,0.060827,False
1461,0.006711,0.000000,0.011628,0.008529,0.007481,0.000000,0.040349,0.014873,0.001855,0.16,0.019465,True
...,...,...,...,...,...,...,...,...,...,...,...,...
570,0.002237,0.000000,0.000000,0.008529,0.002494,0.018293,0.044070,0.021521,0.002968,0.11,0.034063,True
1621,0.013423,0.000000,0.000000,0.010661,0.012469,0.030488,0.044719,0.067958,0.008905,0.04,0.043796,True
1843,0.002237,0.000000,0.000000,0.004264,0.004988,0.000000,0.054056,0.019321,0.002968,0.12,0.024331,False
3535,0.006711,0.000000,0.000000,0.014925,0.007481,0.018293,0.048299,0.060617,0.008534,0.04,0.053528,False


In [52]:
train.label

2186    False
2665    False
6054    False
5373    False
4109    False
        ...  
5051    False
4998    False
2987    False
4871    False
4275    False
Name: label, Length: 8631, dtype: bool

In [57]:
df1 = train[train.iloc[:,-1] == False]

In [58]:
df1

,LOC_BLANK,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CYCLOMATIC_COMPLEXITY,DESIGN_COMPLEXITY,ESSENTIAL_COMPLEXITY,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_ERROR_EST,HALSTEAD_LEVEL,NUM_UNIQUE_OPERATORS,label
2186,0.008949,0.000000,0.008721,0.010661,0.004988,0.000000,0.033803,0.040985,0.004082,0.06,0.031630,False
2665,0.008949,0.000000,0.000000,0.006397,0.002494,0.012195,0.036453,0.065471,0.007050,0.04,0.046229,False
6054,0.013423,0.000000,0.005814,0.021322,0.014963,0.000000,0.036453,0.094165,0.010019,0.03,0.051095,False
5373,0.031320,0.009259,0.020349,0.042644,0.027431,0.036585,0.117554,0.057461,0.020037,0.04,0.051095,False
4109,0.002237,0.000000,0.000000,0.008529,0.002494,0.024390,0.020464,0.045744,0.002597,0.05,0.041363,False
...,...,...,...,...,...,...,...,...,...,...,...,...
5051,0.006711,0.000000,0.000000,0.004264,0.002494,0.000000,0.022816,0.031612,0.002226,0.08,0.034063,False
4998,0.004474,0.009259,0.011628,0.021322,0.007481,0.000000,0.068693,0.045433,0.009276,0.05,0.034063,False
2987,0.002237,0.000000,0.000000,0.000000,0.000000,0.000000,0.038471,0.013319,0.001484,0.18,0.014599,False
4871,0.006711,0.000000,0.008721,0.004264,0.004988,0.000000,0.100706,0.014586,0.004453,0.16,0.019465,False


In [65]:
def sampling(df):
    # create 2 sets of majority and minority samples
    a = df[df.iloc[:,-1] == False]
    b = df[df.iloc[:,-1] == True]
    majority = None
    minority = None
    if a.shape[0] > b.shape[0]:
        print("here")
        majority = a
        minority = b
    else:
        print("there")
        minority = b
        majority = a
    bags = ceil(majority.shape[0],minority.shape[0]) + 2
    print(f"majority cnt :{majority.shape[0]} , minority cnt :{minority.shape[0]} and number of bags: {bags}")
    # print(majority.head())
    # print(minority.head())

    

In [69]:
sampling(train)

here
      LOC_BLANK  LOC_CODE_AND_COMMENT  LOC_COMMENTS  CYCLOMATIC_COMPLEXITY  \
2186   0.008949              0.000000      0.008721               0.010661   
2665   0.008949              0.000000      0.000000               0.006397   
6054   0.013423              0.000000      0.005814               0.021322   
5373   0.031320              0.009259      0.020349               0.042644   
4109   0.002237              0.000000      0.000000               0.008529   

      DESIGN_COMPLEXITY  ESSENTIAL_COMPLEXITY  HALSTEAD_CONTENT  \
2186           0.004988              0.000000          0.033803   
2665           0.002494              0.012195          0.036453   
6054           0.014963              0.000000          0.036453   
5373           0.027431              0.036585          0.117554   
4109           0.002494              0.024390          0.020464   

      HALSTEAD_DIFFICULTY  HALSTEAD_ERROR_EST  HALSTEAD_LEVEL  \
2186             0.040985            0.004082            0